# Forecasting Air Quality with Amazon SageMaker and DeepAR
In this lab, we are going to build an air quality forecasting application using Amazon SageMaker and the DeepAR algorithm. We will walk through how to define the problem, engineer the features, and train, evaluate and deploy the machine learning model. 

## Why is Air Quality important?
Recent bush fire events in Australia is just one reminder of the importance of breathable air. According to the World Health Organization, air pollution is the 4th largest risk factor to human health worldwide, and 90% of the world breathes unhealthy air. Having realiable projections of air quality can help individuals as well as organizations to take steps to mitigate the health affects caused by dangerous air quality levels. For more information on Open AQ's efforts to solve this global issue, visit [Open AQ](https://openaq.org/).

<img src="img/syd_harb_air.jpg" style="width: 600px; border: 15px solid #fff" alt="Sydney Harbour Air During Bushfires"/>


## Problem definition
The first step in any machine learning problem, is to understand the desired outcome.  You need to have a concrete goal to work towards through the entire process of discovery, design, development, deployment and operation. Try to answer these questions up front:

- What will be consuming the predictions?
- How will the predictions be used? 
- How often do predictions need to be made?
- What are the minimum KPI's for the predictions in order for them to be useful?

Throughout your project, you should continuously review that your work is meeting the pre-defined end goals. Defining a clear problem statement up front is critical throughout the process. 

For this problem, the project sponsors have given a detailed use case description:
> *A state environmental protection agency wishes to provide air pollution estimates for particulate matter via a public web page. Predictions need to be made for over 100 locations throughout Australia. The forecast should be updated every day, and needs to show the projected particulate matter 10-micron (pm10) values for the next 2 days on an hourly basis. A range of possible values should be shown. The generated static web page should have a link to a csv file of the predictions as well as a summary visualization. The predicted air quality should be averaged over 24 hours and classified according to the Victorian Air Quality standards for pm10. Healthy and unhealthy air days should be predicted correctly 75% of the time.*

<table>
    <tbody>
        <tr>
            <th>Air quality category</th>
            <th>
            <p><span>PM</span><sub><span>10</span></sub><span>&nbsp;µg/m</span><sup><span>3&nbsp;</span></sup><span>averaged over 1&nbsp;hour&nbsp;</span></p>
            </th>
        </tr>
        <tr>
            <th style="background-color: #64a13c; text-align: center;"><span style="color: white;">Good</span></th>
            <td>&nbsp; Less than 40</td>
        </tr>
        <tr>
            <th style="background-color: #eac51c; text-align: center;">Moderate</th>
            <td>&nbsp; 40–80</td>
        </tr>
        <tr>
            <th style="background-color: #d67900; text-align: center;"><span style="color: white;">Poor</span></th>
            <td>&nbsp; 80–120</td>
        </tr>
        <tr>
            <th style="background-color: #a90737; text-align: center;"><span style="color: white;">   Very poor</span></th>
            <td>&nbsp; 120–240</td>
        </tr>
        <tr>
            <th style="background-color: #50051e; text-align: center;"><span style="color: white;">Hazardous</span></th>
            <td>&nbsp; More than 240</td>
        </tr>
    </tbody>
</table>


Getting such a well defined problem statement is rarely easy. You will need to spend time with the project sponsors to understand what the desired end result is, and how this translates into machine learning requirements. Despite the difficulty in establishing thorough requirements, it's critical we figure this out up front to avoid developing a machine learning model that does not meet clearly defined objectives. 

> There is one thing that is not clear from the problem statement above. What is the mathematical way to express "healthy" versus "unhealthy"? After speaking with the project sponsors, we are told that any pm10 value > 80 is unhealthy.

From the problem statement there are several key points that determine what needs to be built:
- This is a 2-day forecast, with hourly frequency.
- The forecast will be created every hour using batch techniques.
- The probability distribution of predicted pm10 values is needed, not just a point forecast.
- We need to classify a 24-hour rolling average, according to the stated pm10 ranges for "good", "moderate", "poor", "very poor" and "hazardous".
- For evaluation, the percentage of time we correctly predict unhealthy versus healthy is calculated.
- There are no pre-existing benchmarks to beat. Sometimes when developing a new predictive model, it needs to beat an existing system.
- The application will generate a static html page for the visualizations and raw csv files for the predictions.


## General approach
Now that there is a defined problem, here are the steps needed:
1. Find a data source for pm10 values with at least one-hour resolution for Australia.  
2. Explore the data and perform some analysis on its properties.
3. Perform feature engineering to transform the data into training and test data sets.
4. Train a machine learning model with the training data.
5. Infer predictions using the trained model on the test data.
6. Calculate the categorical class for the predictions based on the ranges above.
7. Evaluate the trained model by comparing actuals versus predicted.
8. Create graphs and CSV files of the predictions.
9. Create a machine learning pipeline to automate all of the above.
> **Agility Is Important:** Machine learning projects are not linear. Many of the steps above could require repeating previous steps. For example, the data sources might be missing data, and the problem statement needs to be reworked. Also, the evaluated model might not meet evaluation criteria, so additional data needs to be found. An agile approach that allows for experimentation, failure and redoing steps is required. 

## Data Discovery 
The data used to train the forecasting models needs to be found first. There are many open data sets available, as well as data from your own organizations. One good source of data is [Registry of Open Data on AWS](https://registry.opendata.aws). The registry has a simple search interface that can be used to find data. Now search for "air quality" data sets:

![Registry of Open Data](img/rod_screen_shot.png)

The [OpenAQ](https://registry.opendata.aws/openaq/) data set has per city air quality measurements at hourly frequency, and is a perfect fit for the problem.

![Open AQ](img/openaq_screen_shot.png)

The data is publicly available on S3 and contains many locations and different quality measurements. It needs to be filtered down to only Australia pm10 measurements. By Amazon Athena to query the data, a smaller csv file containing only what is needed can be created.

### Import project dependencies
Before beginning, first import all the python modules, configuration settings and helper functions needed. 

> **Note:** When developing your own projects, it's recommended to separate lower level code out to a python module to make the notebook more readable. This also makes putting the final code into production format easier as it is already partly modularized.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install --upgrade sagemaker geopandas

In [ ]:
from project_dependencies import *

### Create external Athena table
The table used to query against is backed by a publicly readable S3 bucket provided by Open AQ. The data definition language (DDL) query sets the external tables S3 location, and table definition, so it can be queried by Athena. A simple function that uses the boto3 API's to execute operations with Athena and wait for results is used to both create the table and query it. 

> Note: The table definition is based on this [code](https://gist.github.com/jflasher/573525aff9a5d8a966e5718272ceb25a). The code creates an external table in Amazon Athena. To save the OpenAQ organization on data transport costs out of region, run Athena queries in the US East region if possible.

In [ ]:
create_results_uri = athena_create_table('sql/openaq.ddl')

### Query data with Athena
The data manipulation language (DML) query queries the external OpenAQ table for the data required, and places it into an S3 bucket owned by this account. 

> **Try this:** If you would like to modify this lab to be more relevent to where you live, try copying and modifying the sql in the `openaq/sql` file to get data for your country. Not all countries have data. If you are interested in building your own air quality measurement sensor, do a search on "DIY air quality monitoring".

In [ ]:
query_results_uri = athena_query_table('sql/sydney.dml')

> **Try this:** Copy the SQL statement above. Now go to the [Athena console](https://console.aws.amazon.com/athena/home), paste the SQL in the query window and modify it to get the average (use the "avg" function) value of the Carbon Monoxide (code "co") measurements for Melbourne.

## Feature engineering
Feature engineering is the process of transforming raw data into features that can be used for training and testing a machine learning model. Time series data can be stored in unsorted format, can have missing values or may be captured at a higher or lower frequency than what is needed. In addition, machine learning algorithms require the data be put in a standard format for both training and inference. The main steps taken to format the raw data are outlined here.

![Feature Engineering](img/feature_engineering.png)

### Step 1: Read raw data
The Athena query that was previously run places the query results in CSV format in an S3 bucket. The path to the csv object is a combination of a base path defined for all Athena query results and the query id. Using these values, we construct an S3 path, and then read the data into an in-memory panda data frame. Pandas is a data science library that makes it easy to process and explore time series data. Using the pandas csv function the query results can be read directly into an in-memory data frame. 

> **Explore:** For more information on working with Pandas, check out the  [user guide](https://pandas.pydata.org/docs/user_guide/index.html).

In [ ]:
print (f'reading {query_results_uri}')
raw = pd.read_csv(query_results_uri, parse_dates=['timestamp'])
raw.head()

### Step 2: Sort and index by location and time 
Before doing any transformations, the raw data needs to be organized by time and locations. In pandas, we can easily sort by data frame columns, and then create an index for all the categorical columns as well as time. The names of index columns are referred to as "levels" in pandas. After executing the query below, you will notice that the readings are now sorted and grouped by the index levels.

In [ ]:
categorical_levels = ['country', 'city', 'location', 'parameter']
index_levels = categorical_levels + ['timestamp']
indexed = raw.sort_values(index_levels, ascending=True)
indexed = indexed.set_index(index_levels)
indexed.head()

> **Exercise:** Get the mean, minimum or maximum pm10 values for all of the selected data.

### Step 3: Down sample to hourly samples by maximum value
Down sampling combines multiple samples that may fall into the same window of time according to the sampling frequency. Some air quality instruments may measure values more than once and hour. Since we want to predict the peak value in any given hour, we will take the maximum of the values for each hour. Depending on the time series use case, you could also use the mean, first, last and minimum value for down sampling. 

> **Note**: Using max on latitude and longitude columns is acceptable if all entries in a single time series have the same latitude and longitude, which is the case for this data.

In [ ]:
downsampled = indexed.groupby(categorical_levels + [pd.Grouper(level='timestamp', freq='1H')]).max()
downsampled.head()

> **Exercise:** Copy the expression above and modify it to down sample with the mean value.

### Step 4: Back fill missing values
A common problem with time series data is that values are often missing. It is important to determine how many missing values there are in your data and perform filling. 

In order to fill missing values, there are two steps. First, we re-index the data for the desired frequency of 1 hour. This will create a NaN (not a number) entry for any missing values. Once we have the reindexed data, we can then calculate some statistics on the number of NaN values. 

To check for missing values, we filter all non-null columns, group them by location, count them per group, and then describe the summary stats.

In [ ]:
def fill_missing_hours(df):
    df = df.reset_index(level=categorical_levels, drop=True)                                    
    index = pd.date_range(df.index.min(), df.index.max(), freq='1H')
    return df.reindex(pd.Index(index, name='timestamp'))

filled = downsampled.groupby(level=categorical_levels).apply(fill_missing_hours)
filled[filled['value'].isnull()].groupby('location').count().describe()

For this data, are not may not be any missing values. Depending on the country the data was gathered from, there may be more. If there are missing values, the next step is to replace the NaN values with something else. For this case, we will linearly interpolate between the last know values for the pm10 measurement. By also rounding to two decimal places, all of the interpolated values will match the actual values precision. We will also fill any missing latitudes or longitudes with the first value available for each location.

In [ ]:
filled['value'] = filled['value'].interpolate().round(2)
filled['point_latitude'] = filled['point_latitude'].fillna(method='pad')
filled['point_longitude'] = filled['point_longitude'].fillna(method='pad')

filled[filled['value'].isnull()].groupby('location').count().describe()

### Step 5: Create features
Now that we have a contiguous data set at the proper frequency, we can transform the data frame into a format that is required for training and testing. The DeepAR algorithm we will be using to train the machine learning model, requires data in the following format. 

`{start: <start time of series>, target: [v1,v2,v3....], cat: [id1, id2, id3...]}`

For each location, the features are the start time for the time series, a contiguous list of all values, and an optional categorical array. This categorical array is composed of all the id's for country, city, location and measurement type.

> **Explore:** Read more about the [DeepAR feature format](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html#deepar-inputoutput).

#### Aggregate time series into rows
To get to the required format, a single row per time series needs to be created. This is done by aggregating each locations time series values into a single list per row, and by only retaining the timestamp of the first value.

In [ ]:
aggregated = filled.reset_index(level=4)\
    .groupby(level=categorical_levels)\
    .agg(dict(timestamp='first', value=list, point_latitude='first', point_longitude='first'))\
    .rename(columns=dict(timestamp='start', value='target'))

In order to relate each time series to future predictions generated from the trained model, a unique id per time series is needed. Once there is a unique id, we can break the data into a metadata data frame and a features data frame.

In [ ]:
aggregated['id'] = np.arange(len(aggregated))
aggregated.reset_index(inplace=True)
aggregated.set_index(['id']+categorical_levels, inplace=True)
aggregated.head()

#### Transform latitude and longitude to geometry
Since our time series also have a geographical location, we will also keep track of latitude and longitude so we can use it for displaying our results on maps. GeoPandas enables geo searches within pandas data frames, and requires a geometry object which is created by combining the latitude and longitude columns into a single point geometry column. The code below does all of this.

In [ ]:
metadata = gpd.GeoDataFrame(
    aggregated.drop(columns=['target','start']), 
    geometry=gpd.points_from_xy(aggregated.point_longitude, aggregated.point_latitude), 
    crs={"init":"EPSG:4326"}
)
metadata.drop(columns=['point_longitude', 'point_latitude'], inplace=True)

# missing lat/longs in queensland
#metadata.loc[pd.IndexSlice[:,'AU','South West Queensland','Miles Airport',:], 'geometry'] = Point(150.165, -26.809167) 
#metadata.loc[pd.IndexSlice[:,'AU','South West Queensland','Hopeland',:], 'geometry'] = Point(150.6655, -26.8841)

# set geometry index
metadata.set_geometry('geometry')

metadata.head()

#### Create a map plot
The map plot will display a map that contains all the locations for our air quality measurements. We are using the bokeh library to plot the map. We will use this map later to select which locations to view predictions for.

> **Explore:** For more information on plotting with bokeh, check out the [user guide](https://docs.bokeh.org/en/latest/index.html). Other popular plotting libraries include [matplotlib](https://matplotlib.org/) and [plotly](https://plotly.com/). 

In [ ]:
output_notebook()
tile_provider = get_provider(CARTODBPOSITRON)
curdoc().theme = 'light_minimal'

map_df = metadata.to_crs("EPSG:3857").reset_index()
map_source = GeoJSONDataSource(geojson=map_df.to_json(na='drop'))

tooltips = [
    ('Country', '@country'),
    ('City', '@city'),
    ('Location', '@location')
]

map_plot = figure(
    title="Measurement Locations", 
    plot_width=800, plot_height=400, 
    x_axis_type="mercator", y_axis_type="mercator", 
    tooltips=tooltips
)

map_plot.add_tile(tile_provider)
map_circles = map_plot.circle(x="x", y="y", size=5, fill_color="blue", fill_alpha=0.8, source=map_source)
show(map_plot)

> **Exercise:** Try to change the plot size, colour of the location, and add 'parameter' as a tooltip to the above plot.

#### Add categorical feature
The DeepAR model allows time series to be associated by categorical features. This is a key feature, as we are building a single machine learning model to predict air quality for many different locations. The categorical features enable the deep learning model to build an internal representation of how locations are related to each other. To do this, we create a list of ids that represent each of categorical features. The country, city, location, and measurement type will each be codified as an id, and combined into a list for each locations time series.

> **Note:** Since we only are building a model for pm10 values in Australia, only the city and location ids will have multiple values. We could modify the initial query to get multiple measurement types across many countries and train a more comprehensive model using exact the same steps we have walked through.

To get a set of categorical ids, we use the pandas factorize method to generate id's for categorical values. We then apply string to categorical conversion across every row of our aggregated data set.

In [ ]:
level_ids = [level+'_id' for level in categorical_levels]
for l in level_ids:
    aggregated[l], index = pd.factorize(aggregated.index.get_level_values(l[:-3]))
    
aggregated['cat'] = aggregated.apply(lambda columns: [columns[l] for l in level_ids], axis=1)
features = aggregated.drop(columns=level_ids+ ['point_longitude', 'point_latitude'])
features.reset_index(level=categorical_levels, inplace=True, drop=True)
features.head()

### Step 6: Split features into training and test sets
In order to evaluate the final model, the features need to be split into training and test sets. To accurately get an idea of how the model will perform on previously unseen data, time series data should be split according to a cut-off date.

![training and test split by time](img/train_test_split.png)

In [ ]:
train_test_split_date = date.today() - timedelta(days=30)
train = filter_dates(features, None, train_test_split_date, '1H')
train.head()

In [ ]:
test = filter_dates(features, train_test_split_date, None, '1H')
test.head()

## Train DeepAR model
Now that we have our features split into train and test sets, we can train a machine learning model. For this problem, we will use DeepAR, a first party algorithm available in SageMaker. DeepAR is a deep learning algorithm that creates a single machine learning model for multiple related time series. For more information [read the docs](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html).

### Create estimator
To create an estimator, get the image name for the desired algorithm, the execution role and determine the number of training instances and instance type needed. For DeepAR we can use a CPU instance type.

In [ ]:
session = sagemaker.Session()
region = session.boto_region_name
role = sagemaker.get_execution_role() 
image_uri = sagemaker.image_uris.retrieve('forecasting-deepar', region, '1')
output_path = f's3://{bucket_name}/sagemaker/output'

estimator = sagemaker.estimator.Estimator(
        sagemaker_session= session,
        image_uri= image_uri,
        role= role,
        instance_count= 1,
        instance_type='ml.c5.xlarge',
        base_job_name= 'deepar-openaq-demo',
        output_path= output_path
)

### Upload training data to S3
In order to use the training data, we need to get it from in-memory pandas data frames to a location in S3 where SageMaker can use it. We first write all of the train and test set features to a local location. We can then use the SageMaker upload_data method to upload them to S3, and get a reference id that is used later by SageMaker.

In [ ]:
os.makedirs('data', exist_ok=True)
train.to_json('data/train.json', orient='records', lines=True)
test.to_json('data/test.json', orient='records', lines=True) 

data = dict(
    train= session.upload_data(path='data/train.json'),
    test=  session.upload_data(path='data/test.json')
)

### Create a hyperparameter optimization (HPO) job
Machine learning models have tuneable parameters that need to be set prior to training a model. These parameters affect both the time it takes to train a model and the quality metrics of the trained model. Hyperparameter optimization jobs train multiple models across ranges of these parameters to find the best combinations for the given data set. SageMaker HPO uses Bayesian optimization to rapidly find the optimal parameters much faster than a random or grid search. There are two sets of parameters below. The parameters with a range of values will be optimized, and static parameters won't be. For more information on the DeepAR hyper-parameters, visit the [developer guide.](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar_hyperparameters.html)

![hyper-parameter tuning](img/hpo.png)

> Note: HPO does not need to run during every training cycle. Because it is training multiple models, it is time consuming and expensive. Typically, you run HPO once, and then use static hyperparameters until the ML models are no longer meeting evaluation metric goals. Since I have already run a large HPO job to determine the hyperparameters, they are set statically below, and we will only start the HPO job to demonstrate how it operates. If you are training with a different country or measurement type, re-running a full HPO job will give you improved results.

#### Set static hyperparameters

The static parameters are the ones we know to be the best based on previously run HPO jobs, as well as the non-tuneable parameters like prediction length and time frequency that are set according to requirements.

In [ ]:
hpo = dict(
    time_freq= '1H'
    ,early_stopping_patience= 40
    ,prediction_length= 48
    ,num_eval_samples= 10
    ,test_quantiles= quantiles
    
    # not setting these since HPO will use range of values
    #,epochs= 400
    #,context_length= 3
    #,num_cells= 157
    #,num_layers= 4
    #,dropout_rate= 0.04
    #,embedding_dimension= 12
    #,mini_batch_size= 633
    #,learning_rate= 0.0005
)

#### Set hyper-parameter ranges
The hyperparameter ranges define the parameters we want the tuner to search across.

> Explore: Look in the [user guide](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar_hyperparameters.html) for DeepAR and add the recommended ranges for `embedding_dimension` to the below.

In [ ]:
hpo_ranges = dict(
    epochs= IntegerParameter(1, 1000)
    ,context_length= IntegerParameter(7, 48)
    ,num_cells= IntegerParameter(30,200)
    ,num_layers= IntegerParameter(1,8)
    ,dropout_rate= ContinuousParameter(0.0, 0.2)
    ,embedding_dimension= IntegerParameter(1, 50)
    ,mini_batch_size= IntegerParameter(32, 1028)
    ,learning_rate= ContinuousParameter(.00001, .1)
)

#### Create and start the HPO job
Once we have the HPO tuner defined, the fit method is called. This will trigger the launching of the training instances. When creating the tuner, an objective metric is needed to compare each model it trains. For this model, the tuner will find the model with the minimal final loss.

In [ ]:
estimator.set_hyperparameters(**hpo)

hpo_tuner = HyperparameterTuner(
    estimator= estimator, 
    objective_metric_name= 'train:final_loss',
    objective_type='Minimize',
    hyperparameter_ranges= hpo_ranges,
    max_jobs=2,
    max_parallel_jobs=1
)

hpo_tuner.fit(data, wait=False)
display_hpo_tuner_advice(hpo_tuner)

The tuning job will run for a few hours, as it will be training several machine learning models, each of which can take as long as 2 hours. Instead of waiting, we will use a model that created from a previous HPO job that ran for two days, and compared 150 models. The code below will stop the running HPO job, and then copy this model artifact to a location in S3 to simulate the final output of an HPO job. 

> **Note:** If you are running this lab and want to optimize on a different set of data be sure to comment out the code below to allow your hpo job to complete, and give you a customized model. Warning! This will take some time to complete!

In [ ]:
hpo_tuner.stop_tuning_job()
hpo_tuner = None

In order to wait for the HPO job to complete, call wait on the tuner. The HPO job will take many hours to complete if it was not stopped above.

In [ ]:
if hpo_tuner:
    hpo_tuner.wait()

### Train model on complete data set
The HPO job we ran above was not trained on the most recent data. Since we now have the best hyper-parameters, we will retrain a model on the most up to date data so any current patterns in air quality will be learned by the model. In production, we would normally skip over the hyperparameter optimization job, and just retrain our model on a daily basis prior to creating inferences. If the quality of our forecasts in production starts to go down, or we are aware of big changes to the data sources, for example if new countries are added, we would rerun the hpo job to get new hyper-parameters.

![model training](img/training.png)

In [ ]:
# use full set of data for training
features.to_json('data/all_data.json', orient='records', lines=True)
data = dict(train= session.upload_data(path='data/all_data.json'))

hpo = dict(
    time_freq= '1H'
    ,early_stopping_patience= 40
    ,prediction_length= 48
    ,num_eval_samples= 10
    ,test_quantiles= quantiles    
    ,epochs= 400
    ,context_length= 3
    ,num_cells= 157
    ,num_layers= 4
    ,dropout_rate= 0.04
    ,embedding_dimension= 12
    ,mini_batch_size= 633
    ,learning_rate= 0.0005
)

estimator.set_hyperparameters(**hpo)
estimator.fit(data, wait=False)
training_job = estimator.latest_training_job

display_training_job_advice(training_job)

> Note: If you want to train on a different set of locations or parameters be sure to comment out the code below to allow your training job to complete, and give you a customized model. Warning! This will take some time to complete!

In [ ]:
training_job.stop()
training_job = None

### Deploy the best model as an endpoint
A trained model exists as a data structure within an archived file. In order to create predictions with it, we need to deploy it as an endpoint. An endpoint can be called with feature data and will return the predictions. The feature data for our model is the timestamp, the last 3 hours of readings (the context length) and the categorical information describing the location of a measurement. Since we stopped training so as to save time, we will use a previously trained model, upload it to S3 and create a model object from it.

In [ ]:
endpoint_name = None
if endpoint_name is None: # if we dont already have a deployed endpoint
    if training_job: # if we are waiting for training to complete
        training_job.wait()
        model = estimator.create_model()
    
    else: # if we want to use a previously trained model
        s3.upload_file('model/model.tar.gz', bucket_name, 'sagemaker/model/model.tar.gz')
        model = Model(
            image_uri, 
            model_data=f's3://{bucket_name}/sagemaker/model/model.tar.gz', 
            role=role, 
            name='openaq-previously-trained-model', 
            sagemaker_session=session
        )
        
    model.deploy(initial_instance_count=1, instance_type='ml.c5.xlarge', wait=False)
    endpoint_name = model.endpoint_name
    
display_endpoint_advice(session, endpoint_name, wait=True)

Endpoint deployment can take a few minutes to start as SageMaker needs to start up the instance. As an alternative using endpoints, it is also possible to use a batch transformer to create inferences for data located in S3. This is out of scope for this tutorial.

## Create inferences (predictions)

Now that we have a trained model, we need to evaluate it using the holdout data. Using this holdout data is only needed when you first are creating the model in order to get an idea of how the model will perform against new data in production. After the model is running in production, it is better to always retrain the model on all available data, and then monitor model performance over time against a trailing set of historical data.

### Generate test sets to predict
To get an idea of how the model performs, we will create predictions on a 12 hour rolling basis for all of the  locations, and then graph and compare them to the actuals. The method below generates the features from the hold out set to do this.

In [ ]:
ten_days_ago =  date.today() - timedelta(days=10)
test_dates = pd.date_range(ten_days_ago, periods=216, freq='1H')
tests = get_tests(test, test_dates, '1', 3, 48)
tests.head()

### Test the endpoint
From the above, you can see that we will need to call our endpoint 4060 times for each of our tests, as we are back testing every hour, across all locations for the previous 10 days. 
Before we call the endpoint with all of the tests we have generated, let's first try calling it for just one location and time. The request passes in an array of features, one for each location, as well as configuration settings.

> **Try this:** Modify the request to get a different quantile, or the predictions for a different test set.

In [ ]:
predictor = Predictor(
    endpoint_name, 
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer()
)

features = tests[['start','target','cat']].iloc[0].to_dict()
json.dumps(predictor.predict({
    'instances': [features]
    ,'configuration': {
        'num_samples': 20
        ,'output_types': ['quantiles']
        ,'quantiles': ['0.5']
    }
}))

### Predict all the values
The predict function below will call the endpoint with multiple processes in parallel to speed up the time. This will still take several minutes as we will be back testing all locations in Sydney for the last month. 

> **Try this:** Look at the endpoint within the console and examine the monitoring section. Watch as the CPU, memory and disk increase while under load. You can also improve the processing time by editing the configuration and adding auto scaling. Currently the endpoint only has a single endpoint, but can be expanded to more.

In [ ]:
predictions = predict(predictor.endpoint_name, tests, quantiles) 
predictions.head()

### Evaluate the model
Now that we have all the predictions, let's evaluate how close to the original values they are. For evaluation, we will calculate the mean absolute percentage error (MAPE). This value is average percentage difference of the predicted values versus actuals. 

![MAPE formula](https://wikimedia.org/api/rest_v1/media/math/render/svg/961e2d315e7269f820104c7b4b422f840104be2c)

There are other metrics that can be used, but for demonstration purposes, we will use MAPE. One issue with MAPE, is that when the actual values are close to zero, the calculated errors can be very high or infinite due to divide by zero. Our data has this issue, so we will use a variation on MAPE and divide by the average value instead of the absolute value.

In [ ]:
plot_error(test, predictions, horizon=12)
plot_error(test, predictions, horizon=24)
plot_error(test, predictions, horizon=48)

From the plot, you can see that 30% and 40% quantiles have the best MAPE, where the average error is around 60% of the expected. 


### Add moving average
According to the problem statement, we need to use the average pm10 value over 24 hours. We will use the moving average of the mean quantile (0.5 quantile) to determine the predicted pm10 value. The mean quantile signifies that 50% of the time the actual values will be above the predicted value. If we wanted our predictions to be less likely to go over our threshold "bad" value, we could pick a lower quantile. For example, a '0.4' quantile would mean 60% of actual values would be above the predictions, and '0.3' quantile would mean 70% of actual values would be above the predictions.
> **Try this:** Change the quantile below to make the predictions less likely to flag a bad air quality event

In [ ]:
actuals = filter_dates(
    test, 
    predictions['start'].min(), 
    predictions['start'].max(), 
    '1H'
).drop(columns=['cat'])
test

In [ ]:
ma = predictions['0.4'].apply(moving_average)
pd.DataFrame(ma.rename('ma')).head()

### Calculate predicted air quality events 
According to the problem statement, we want to flag anytime the average pm10 value exceeds the "good" range of  values. Anytime the moving average exceeds 40, we will mark that time period as a bad air quality event.
> **Try this:** Change the threshold value to only mark events that are worse than "moderate".

In [ ]:
predictions['events'] = events = ma.apply(lambda x: x > 40)
counts = pd.DataFrame(events.apply(sum).rename('count'))
counts[counts['count']>0]

### Calculate actual air quality events
Now that we have the predictions, we still need the actuals to compare them to. We will repeat all of the above logic on the actual values, so we can compare to the predicted.

In [ ]:
actuals = filter_dates(
    test, 
    predictions['start'].min(), 
    predictions['start'].max(), 
    '1H'
).drop(columns=['cat'])
actuals['ma'] = ma = actuals['target'].apply(moving_average)
actuals['events'] = events = ma.apply(lambda x: x > 80)
counts = pd.DataFrame(events.apply(sum).rename('count'))
counts[counts['count']>0]

## Compare historical predictions against actuals
Now that we have the metadata, features and predictions,  we will combine them all into an analytics GUI. All the logic has been abstracted into a function that loads the data into a browser indexDB, and then uses javascript callbacks to change the start of the prediction time. As you move the slider, it shows what quantiles the model predicts based on the measuremnts previous to that time. By looking at the graphs, you can see how well the model does at predicting the next 48 hours.

In [ ]:
show(
    create_analysis_chart(
        metadata, 
        actuals, 
        predictions
    )
)

## What's next?
In this lab we have walked through the steps of gathering data with Athena queries, feature engineering, training, and evaluation. **This notebook is still not production ready code!** In order to develop this into a production system, we need to create a workflow that can be triggered on a repeating basis. 

To learn how to do this with SageMaker pipelines, batch transforms and step functions, continue on to the next section of this lab.<br>
**[Continue with machine learning pipeline creation](./02_manual_ml_pipeline_creation_for_air_quality_forecasting.ipynb)**

<br>

If you are interested in learning more about other areas of time series forecasting, these are additional labs are also available:

#### [Predicting electricity production with GluonTS](../2_Predict_electricity_demand_with_the_GluonTS_and_SageMaker_custom_containers/00_predict_electricity_demand_with_the_gluonts_library.ipynb)
Learn about some of the other time series algorithms that are available in GluonTS such as Prophet and ARIMA, and also how to deploy them in a SageMaker custom container.

#### [Automate sales projections with Amazon Forecast](../3_Automate_sales_projections_with_Amazon_Forecast/1_prepare_dataset.ipynb)
Learn how Amazon Forecast, a managed forecasting service, can take care of the heavy lifting of feature engineering, training and testing.  

#### [Time series data and anomaly detection](../4_Anomaly_detection_with_RCF_and_DeepAR/1.EDA-and-DataPrep.ipynb)
Learn how to build an anomaly detector for time series data with random cut forest (RCF) and DeepAR.

## Image Attributions
<p style="font-size: 0.9rem;font-style: italic;"><img style="display: block;" src="https://live.staticflickr.com/65535/49246056803_0a0bae48cf_b.jpg" border="1px" solid="#ddd" border-radius="4px" padding="5px" width="150px" alt="P1210668"><a href="https://www.flickr.com/photos/37912374670@N01/49246056803">"P1210668"</a><span> by <a href="https://www.flickr.com/photos/37912374670@N01">acb</a></span> is licensed under <a href="https://creativecommons.org/licenses/by-nc-sa/2.0/?ref=ccsearch&atype=html" style="margin-right: 5px;">CC BY-NC-SA 2.0</a><a href="https://creativecommons.org/licenses/by-nc-sa/2.0/?ref=ccsearch&atype=html" target="_blank" rel="noopener noreferrer" style="display: inline-block;white-space: none;margin-top: 2px;margin-left: 3px;height: 22px !important;"><img width="15px"style="height: inherit;margin-right: 3px;display: inline-block;" src="https://search.creativecommons.org/static/img/cc_icon.svg" /><img width="15px" style="height: inherit;margin-right: 3px;display: inline-block;" src="https://search.creativecommons.org/static/img/cc-by_icon.svg" /><img width="15px" style="height: inherit;margin-right: 3px;display: inline-block;" src="https://search.creativecommons.org/static/img/cc-nc_icon.svg" /><img width="15px" style="height: inherit;margin-right: 3px;display: inline-block;" src="https://search.creativecommons.org/static/img/cc-sa_icon.svg" /></a></p>